This jupyter notebook contains the code for face morphing using a trained Convolutional Mesh Autoencoder. The architecture is based on the paper [Generating 3D faces using Convolutional Mesh
Autoencoders](https://coma.is.tue.mpg.de/) and is trained on 44 neutral example faces. As template the mesh *headtemplate_noneck_lesshead_4k.obj* was used.

![Face morphing with Autoencoder](../results/faceModel/example.gif)

In [11]:
import igl
import numpy as np
import meshplot as mp
from scipy.sparse.linalg import spsolve
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib notebook

from numpy import genfromtxt
from model import Decoder, Encoder, Downscale
import torch
from torch_geometric.io import write_off, read_obj
from torch_geometric.transforms import FaceToEdge

if torch.cuda.is_available():
        dev = "cuda:0"
else:
        dev = "cpu"
device = torch.device(dev)

addEdges = FaceToEdge(remove_faces=False)

# Downsampled versions of the headtemplate_noneck_lesshead_4k.obj 
mesh0 = read_obj("./recon_meshes/mesh0.obj")
mesh1 = read_obj("./recon_meshes/mesh1.obj")
mesh2 = read_obj("./recon_meshes/mesh2.obj")
mesh3 = read_obj("./recon_meshes/mesh3.obj")
mesh4 = read_obj("./recon_meshes/mesh4.obj")
v1 = genfromtxt("./recon_meshes/V1.txt")
v2 = genfromtxt("./recon_meshes/V2.txt")
v3 = genfromtxt("./recon_meshes/V3.txt")
v4 = genfromtxt("./recon_meshes/V4.txt")

The output reconstruction of the faces using the trained model is a bit noisy therefore the output is smoothed after reconstruction: 

In [2]:
def smooth(v, f):
    l = igl.cotmatrix(v, f)
    n = igl.per_vertex_normals(v, f)*0.5+0.5


    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_BARYCENTRIC)
    s = (m - 10*l)
    b = m.dot(v)
    v = spsolve(s, m.dot(v))
    vs=v
    return vs

Define decoder and encoder checkpoints that should be loaded. There are two trained models at the moment *decoder_44faces.pth/encoder_44faces.pth* and *decoder_44faces_lr5e-4.pth/encoder_44faces_lr5e-4.pth*. Both work well while the second one was trained for a longer time with a smaller learning rate wich improved the results slightly:

In [ ]:
decoder_checkpoint = "ckt/decoder_44faces_lr5e-4.pth"
encoder_checkpoint = "ckt/encoder_44faces_lr5e-4.pth"

In [3]:
def decode(code=[0.,0.,0.,0.,0.,0.,0.,0.]):

    tensor_code = torch.tensor(code).to(device)

    downscaler = Downscale(mesh0, mesh1, mesh2, mesh3, mesh4, v1, v2, v3, v4, device)
    decoder = Decoder(downscaler).to(device)
    decoder.load_state_dict(torch.load(decoder_checkpoint))

    reconstruction = decoder(tensor_code)
    vertices = reconstruction.detach().cpu().numpy()

    return vertices

In [4]:
def encode(pathtomesh):
    
    target = read_obj(pathtomesh).to(device)
    target = addEdges(target)
    target.x = target.pos
    
    downscaler = Downscale(mesh0, mesh1, mesh2, mesh3, mesh4, v1, v2, v3, v4, device)
    encoder = Encoder(downscaler).to(device)
    encoder.load_state_dict(torch.load(encoder_checkpoint))

    code = encoder(target)
    
    return code.detach().cpu().numpy()

The following code encodes the the template to get a vector of size 8 representing the neutral face.

In [5]:
code = encode("./recon_meshes/mesh0.obj")
print(code)

[-0.02449566 -0.08304243  1.4615785   0.13484523  0.4690101  -0.30835578
  0.14531352  0.17747006]


Now the parameters can be edited to change the face interactively in the meshplot. 

In [10]:
v, f = igl.read_triangle_mesh("./recon_meshes/mesh0.obj")
v=smooth(decode(code), f)
p = mp.plot(v,f)

@interact(param0=(-2., 2.), param1=(-2., 2.), param2=(-2., 2.), param3=(-2., 2.), param4=(-2., 2.), param5=(-2., 2.), param6=(-2., 2.), param7=(-2., 2.))
def change(param0=(-2., 2.), param1=(-2., 2.), param2=(-2., 2.), param3=(-2., 2.), param4=(-2., 2.), param5=(-2., 2.), param6=(-2., 2.), param7=(-2., 2.)):
    res = code.copy()
    res += [param0, param1, param2, param3, param4, param5, param6 ,param7]
    p.update_object(vertices=smooth(decode(res),f))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1754722…

interactive(children=(FloatSlider(value=0.0, description='param0', max=2.0, min=-2.0), FloatSlider(value=0.0, …

In the next part two faces can be loaded and then morphed between them by interpolating their representative code of 8 numbers. (*In this case two new faces were loaded to see if the model generalizes well, but it doesn't know what to do with them and generates nearly the same as the template, so training on only 44 faces probably isn't enough*)

In [7]:
facecode1 = encode("../data/faces_warped/netural_henry.obj")
facecode2 = encode("../data/faces_warped/neutral_julian.obj")

v = decode(facecode1)
vs = smooth(v,f)
p1 = mp.plot(vs,f)

@interact(morph = (0.))
def change(morph = (0., 1.)):
    res = facecode1 * (1-morph) + facecode2 * morph
    vs = smooth(decode(res), f)
    p1.update_object(vertices=vs)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0867919…

interactive(children=(FloatSlider(value=0.0, description='morph', max=1.0), Output()), _dom_classes=('widget-i…